# Basic Postflight
Basic structure and skeleton for postflight scripts.

4/15/2020


In [ ]:
import json #unneeded
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib import pyplot
import pandas as pd
import numpy as np

## Import Data
Basic import functions are defined here. 



In [ ]:
def import_json(filename):
    """ Returns a python dataframe using json data from <filename> """
    data = pd.read_json(filename)
    print('JSON data for ', filename)
    print(data.head(5))
    return data


def import_csv(filename):
    """ Returns a python dataframe using csv data from <filename> """
    data = pd.read_csv(filename)
    print('CSV data for ', filename)
    print(data.head(5))
    return data

In [ ]:
print('You can use the .__doc__ function to get more information about a function. ' \
      'E.g. >import_json.__doc')
print(import_json.__doc__ )
print()
piper_table = import_json('n4023f_stamprun.json', )
print()
flightsim_table = import_csv("sample_flight_sim.csv")

#flightsim_table.head()

In [ ]:
timeList = []
for timestamp in flightsim_table.Timestamp:
    timeList.append(pd.to_datetime(timestamp, format='%H:%M:%S'))

flightsim_time_diff = []
for i in range(len(timeList)-1):
    timeDelta = timeList[i+1] - timeList[i]
    flightsim_time_diff.append(timeDelta.total_seconds())

flightsim_time_diff.append('NaN')
flightsim_table['timeDelta'] = flightsim_time_diff

## Initial Data Exploration
Look at the two plots below:
1. What is being plotted?
2. What does this show us?
3. Why is this useful?

To do
1. Create a function, flightsim_check, that conducts multiple checks on the data and reports this to the user (DRJ)
2. Create a function, piper_check, that conducts multiple checks on the data and reports this to the user (TMB)

For 1 and 2, explore multiple methods to present the data checks

3. Create functions, flightsim_pos_plot and flightsim_or_plot, that plot position (3 plots on 1 page), and orientation (3 plots on 1 page) for flightsim data (DRJ)
4. Create functions, piper_pos_plot and piper_or_plot, that plot position (3 plots on 1 page), orientation (course) for piper data (TMB)

In [ ]:
#pd.plotting.deregister_matplotlib_converters()
plt.plot(flightsim_table.Timestamp)


In [ ]:
#pd.plotting.register_matplotlib_converters() # needed for matplotlib-pandas bug
plt.plot(piper_table.Timestamp)

In [ ]:
def flightsim_check(flightsim_data):
    '''reports info about the quality of the data '''
    flightsim_data_mod = flightsim_data.copy()

    #flightsim_data_mod['timeDelta']=flightsim_data_mod.Timestamp-flightsim_data_mod.Timestamp.shift(1)
    print(flightsim_data_mod.timeDelta.describe())
    print()
    # checks and reports data type but doesn't recognize them as float64 even though the dtypes command lists them as float64
    for i in range(len(flightsim_data_mod.axes[1])-1):
        if flightsim_data_mod.dtypes[i]=='float64':
            print('data type is', flightsim_data_mod.dtypes[i])
        elif flightsim_data_mod.dtypes[i]=='datetime64[ns]':
            print('data type is',flightsim_data_mod.dtypes[i])
        else:
            print('incorrect data type',flightsim_data_mod.dtypes[i])
    
    print(flightsim_data_mod.head(7))
    return flightsim_data_mod
    
    #check for and replace missing data with nan       
    #flightsim_data = flightsim_data.replace(0, np.NaN)
    

In [ ]:
flightsim_check(flightsim_table)

In [ ]:
def flightsim_pos_plot(flightsim_data):
    '''plots position for altitude, longitude and latitude for flight simulation csv data'''
    plt.figure(figsize=(8,6))
    plt.title('Lat Lon and Alt time aligned')
    plt.subplot(311)
    plt.plot(flightsim_data.Time_Serial,flightsim_data.Altitude)
    plt.ylabel('Altitude')
    plt.subplot(312)
    plt.plot(flightsim_data.Time_Serial,flightsim_data.Longitude)
    plt.ylabel('Longitude')
    plt.subplot(313)
    plt.plot(flightsim_data.Timestamp,flightsim_data.Altitude)
    plt.ylabel('Latitude')
    plt.show()

In [ ]:
flightsim_pos_plot(flightsim_table)

In [ ]:
def flightsim_ori_plot(flightsim_data):
    '''plots orientation for heading, roll and pitch for flight simulation csv data'''
    plt.figure(figsize=(8,6))
    plt.title('Heading Roll and Pitch time aligned')
    plt.subplot(311)
    plt.plot(flightsim_data.Time_Serial,flightsim_data.Heading)
    plt.ylabel('Heading')
    plt.subplot(312)
    plt.plot(flightsim_data.Time_Serial,flightsim_data.Roll)
    plt.ylabel('Roll')
    plt.subplot(313)
    plt.plot(flightsim_data.Timestamp,flightsim_data.Pitch)
    plt.ylabel('Pitch')
    plt.show()

In [ ]:
flightsim_ori_plot(flightsim_table)

In [ ]:
#pd.plotting.deregister_matplotlib_converters()
plt.plot(flightsim_table.Time)

In [ ]:
#pd.plotting.register_matplotlib_converters() # needed for matplotlib-pandas bug
plt.plot(piper_table.Timestamp)

In [ ]:
# 2
# what to check: proper data types, describe function
def piper_check(data):
    '''Returns any issues dicovered in the data types, 
    timing, missing data, and vertical speed if found,
    returns a statement that the data is valid if no 
    problems found.'''
    #checking proper data types
    type1 = (data.Latitude.dtypes == 'float64')
    type2 = (data.Longitude.dtypes == 'float64')
    type3 = (data.Course.dtypes == 'float64')
    type4 = (data.Altitude.dtypes == 'float64')
    type5 = (data.Speed.dtypes == 'float64')
    type6 = (data.Timestamp.dtypes == 'datetime64[ns, UTC]')
    if not(type1 and type2 and type3 and type4 and type5 and type6):
        return print('Data is not valid, invalid data type')
    
    #checking to see if there are any significant breaks in time
    check_Gaps = 0
    piper_time_diff =[]
    for i in range(len(data.Timestamp)-1):
        piper_time_diff.append(pd.to_timedelta(piper_table.Timestamp[i+1] - piper_table.Timestamp[i]).total_seconds())
    
    index = 0
    piper_time_stdv = np.std(piper_time_diff)
    for check_Step in piper_time_diff:
        if (check_Step < 0) or (check_Step >= 2*piper_time_stdv):  #identify specific reason for gap size
            check_Gaps = check_Gaps + 1
            print('Warning, data has significant time gap of', check_Step, 'seconds at', data.Timestamp[index])
        index = index + 1
    
    if 'NaN' in data:
        print('Missing data, NaN is present')
    
    max_Vertical_Speed = 0
    for i in range(len(data.Altitude)-1):
        altitude_Diff = data.Altitude[i+1] - data.Altitude[i]
        alt_diff_time = piper_time_diff[i]
        vertical_speed = altitude_Diff / alt_diff_time
        if abs(vertical_speed) > max_Vertical_Speed:
            max_Vertical_Speed = vertical_speed
    print('The maximum vertical speed from one point in time to another is', max_Vertical_Speed, 'ft/s')
    
    if check_Gaps != 0:
        return
    return print('Data is valid')

# for check in
piper_check(piper_table)

In [ ]:
def piper_pos_plot(data):
    timeList = []
    
    for i in range(len(data.Timestamp)):
        timeList.append(data.Timestamp[i].to_pydatetime())
        
    fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True)
    fig.set_size_inches(18.5, 10.5)
    fig.suptitle('Latitude, Longitude, and Altitude vs Time')
    ax1.plot(timeList,data.Latitude)
    ax1.set_ylabel('Latitude (°)')
    ax2.plot(timeList,data.Longitude)
    ax2.set_ylabel('Longitude (°)')
    ax3.plot(timeList,data.Altitude)
    ax3.set_xlabel('Timestamp')
    ax3.set_ylabel('Altitude (ft)')
    plt.show()


piper_pos_plot(piper_table)

In [ ]:
def piper_or_plot(data):
    timeList = []
    
    for i in range(len(data.Timestamp)):
        timeList.append(data.Timestamp[i].to_pydatetime())
        
    fig, ax = plt.subplots()
    fig.set_size_inches(18.5, 10.5)
    ax.plot(timeList,data.Course)
    ax.set_xlabel('Timestamp')
    ax.set_ylabel('Course (°N)')
    fig.suptitle('Course over Time')
    plt.show()

piper_or_plot(piper_table)

## Process Data

For consistent (i.e. format doesn't change) files you read in, it may be useful to create functions to process the files, add derived parameters, and do other standard transformations. In this case, we'll separate the read functions from the processing functions.

Questions/Checks:
1. What is the purpose of this function definition?
2. What happens if we don't use .copy()?
3. Why would we want this behavior?
4. Read through the code and comment on anything notable

To do:
1. Add similar calculations for velocity, other derivatives. Add documentation and docstring (TMB)
2. Create a process_piper_data function that calculates derived parameters. Add documentation and docstring (DRJ)


In [ ]:
ts_diff = []
lat_diff = []
long_diff = []
alt_diff = []
orient_diff = []
lat_vel = []
long_vel = []
alt_vel = []
orient_vel = []


for i in range(len(piper_table) - 1):
    # time difference
    timedelta = pd.to_timedelta(piper_table.Timestamp[i+1] - piper_table.Timestamp[i]).total_seconds()
    ts_diff.append(timedelta)
    
    # position difference
    lat_diff.append(piper_table.Latitude[i+1] - piper_table.Latitude[i])
    long_diff.append(piper_table.Longitude[i+1] - piper_table.Longitude[i])
    alt_diff.append(piper_table.Altitude[i+1] - piper_table.Altitude[i])
    orient_diff.append(piper_table.Course[i+1] - piper_table.Course[i])

    
for i in range(len(ts_diff)):
    # velocity
    lat_vel.append(lat_diff[i] / ts_diff[i])
    long_vel.append(long_diff[i] / ts_diff[i])
    alt_vel.append(alt_diff[i] / ts_diff[i])
    orient_vel.append(orient_diff[i] / ts_diff[i])


lat_vel.append('NaN')
piper_table.insert(6,"Latitude_Velocity", lat_vel)
long_vel.append('NaN')
piper_table.insert(7,"Longitude_Velocity", long_vel)
alt_vel.append('NaN')
piper_table.insert(8,"Altitude_Velocity", alt_vel)
orient_vel.append('NaN')
piper_table.insert(9,"Course_Velocity", orient_vel)

# speed in direction of motion & 2-d plot

In [ ]:
def piper_vel_plot(data):
    timeList = []
    
    for i in range(len(data.Timestamp)):
        timeList.append(data.Timestamp[i].to_pydatetime())
        
    fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True)
    fig.set_size_inches(18.5, 10.5)
    fig.suptitle('Velocity of Latitude, Longitude, and Altitude vs Time')
    ax1.plot(timeList,data.Latitude_Velocity)
    ax1.set_ylabel('Latitude Velocity (°/sec)')
    ax2.plot(timeList,data.Longitude_Velocity)
    ax2.set_ylabel('Longitude Velocity (°/sec)')
    ax3.plot(timeList,data.Altitude_Velocity)
    ax3.set_xlabel('Timestamp')
    ax3.set_ylabel('Altitude Velocity (ft/sec)')
    plt.show()

piper_vel_plot(piper_table)

In [ ]:
ts_diff_vel = ts_diff
lat_diff_vel = 0
long_diff_vel = 0
alt_diff_vel = 0
orient_diff_vel = 0
lat_accel = []
long_accel = []
alt_accel = []
orient_accel = []


for i in range(len(ts_diff_vel) - 1):
    # velocity difference
    lat_diff_vel = piper_table.Latitude_Velocity[i+1] - piper_table.Latitude_Velocity[i]
    long_diff_vel = piper_table.Longitude_Velocity[i+1] - piper_table.Longitude_Velocity[i]
    alt_diff_vel = piper_table.Altitude_Velocity[i+1] - piper_table.Altitude_Velocity[i]
    orient_diff_vel = piper_table.Course_Velocity[i+1] - piper_table.Course_Velocity[i]
    
    # acceleration
    lat_accel.append(lat_diff_vel / ts_diff_vel[i])
    long_accel.append(long_diff_vel / ts_diff_vel[i])
    alt_accel.append(alt_diff_vel / ts_diff_vel[i])
    orient_accel.append(orient_diff_vel / ts_diff_vel[i])


lat_accel.append('NaN')
lat_accel.append('NaN')
piper_table.insert(10,"Latitude_Acceleration", lat_accel)
long_accel.append('NaN')
long_accel.append('NaN')
piper_table.insert(11,"Longitude_Acceleration", long_accel)
alt_accel.append('NaN')
alt_accel.append('NaN')
piper_table.insert(12,"Altitude_Acceleration", alt_accel)
orient_accel.append('NaN')
orient_accel.append('NaN')
piper_table.insert(13,"Course_Acceleration", orient_accel)

In [ ]:
def piper_accel_plot(data):
    timeList = []
    
    for i in range(len(data.Timestamp)):
        timeList.append(data.Timestamp[i].to_pydatetime())
        
    fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True)
    fig.set_size_inches(18.5, 10.5)
    fig.suptitle('Acceleration of Latitude, Longitude, and Altitude vs Time')
    ax1.plot(timeList,data.Latitude_Acceleration)
    ax1.set_ylabel('Latitude Acceleration (°/sec^2)')
    ax2.plot(timeList,data.Longitude_Acceleration)
    ax2.set_ylabel('Longitude Acceleration (°/sec^2)')
    ax3.plot(timeList,data.Altitude_Acceleration)
    ax3.set_xlabel('Timestamp')
    ax3.set_ylabel('Altitude Velocity (ft/sec^2)')
    plt.show()

piper_accel_plot(piper_table)

In [ ]:
#1
def process_flightsim_data(flightsim_data):
    '''Returns a pandas dataframe with all the data from the the flight
    dataframe and the speeds and accelerations of altitude, longitude,
    latitude, roll, pitch, and heading'''
    flightsim_modded = flightsim_data.copy() #copies data, used to prevent multiple copies of same data if code used multiple times
    
    #adds new columns containing speeds
    flightsim_modded['VerticalSpeed'] = (flightsim_data.Altitude - flightsim_data.Altitude.shift(1))/(flightsim_data.Time - flightsim_data.Time.shift(1))
    flightsim_modded['LongitudeSpeed'] = (flightsim_data.Longitude - flightsim_data.Longitude.shift(1))/(flightsim_data.Time - flightsim_data.Time.shift(1))
    flightsim_modded['LatitudeSpeed'] = (flightsim_data.Latitude - flightsim_data.Latitude.shift(1))/(flightsim_data.Time - flightsim_data.Time.shift(1))
    flightsim_modded['RollSpeed'] = (flightsim_data.Roll - flightsim_data.Roll.shift(1))/(flightsim_data.Time - flightsim_data.Time.shift(1))
    flightsim_modded['PitchSpeed'] = (flightsim_data.Pitch - flightsim_data.Pitch.shift(1))/(flightsim_data.Time - flightsim_data.Time.shift(1))
    flightsim_modded['HeadingSpeed'] = (flightsim_data.Heading - flightsim_data.Heading.shift(1))/(flightsim_data.Time - flightsim_data.Time.shift(1))
    #adds new columns containing accelerations, uses flightsim_modded because speed isn't present in flight_simdata
    flightsim_modded['VerticalAcceleration'] = (flightsim_modded.VerticalSpeed - flightsim_modded.VerticalSpeed.shift(1))/(flightsim_data.Time - flightsim_data.Time.shift(1))
    flightsim_modded['LongitudeAcceleration'] = (flightsim_modded.LongitudeSpeed - flightsim_modded.LongitudeSpeed.shift(1))/(flightsim_data.Time - flightsim_data.Time.shift(1))
    flightsim_modded['LatitudeAcceleration'] = (flightsim_modded.LatitudeSpeed - flightsim_modded.LatitudeSpeed.shift(1))/(flightsim_data.Time - flightsim_data.Time.shift(1))
    flightsim_modded['RollAcceleration'] = (flightsim_modded.RollSpeed - flightsim_modded.RollSpeed.shift(1))/(flightsim_data.Time - flightsim_data.Time.shift(1))
    flightsim_modded['PitchAcceleration'] = (flightsim_modded.PitchSpeed - flightsim_modded.PitchSpeed.shift(1))/(flightsim_data.Time - flightsim_data.Time.shift(1))
    flightsim_modded['HeadingAcceleration'] = (flightsim_modded.HeadingSpeed - flightsim_modded.HeadingSpeed.shift(1))/(flightsim_data.Time - flightsim_data.Time.shift(1))
    return flightsim_modded

flightsim_mod = process_flightsim_data(flightsim_table)
#flightsim_mod.head(20)
#flightsim_table.head(10)

In [ ]:
def flightsim_vel_plot(data):
    '''Takes in the data and generates the velocity plots of latitude,
    longitude, and altitude, requires latitude, longitude, altitude and 
    time to be in a pandas dataframe, with the columns labeled Latitude, 
    Longitude, Altitude, and Time respectively'''
    
    # gets the velocity data
    vel_data = process_flightsim_data(data)
    
    # plots the velocity data
    fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True)
    fig.set_size_inches(18.5, 10.5)
    fig.suptitle('Velocity of Latitude, Longitude, and Altitude vs Time')
    ax1.plot(vel_data.Time,vel_data.LatitudeSpeed)
    ax1.set_ylabel('Latitude Velocity (°)')
    ax2.plot(vel_data.Time,vel_data.LongitudeSpeed)
    ax2.set_ylabel('Longitude Velocity (°)')
    ax3.plot(vel_data.Time,vel_data.VerticalSpeed)
    ax3.set_xlabel('Time')
    ax3.set_ylabel('Altitude Velocity (ft)')
    plt.show()


flightsim_vel_plot(flightsim_mod)

In [ ]:
def flightsim_accel_plot(data):
    '''Takes in the data and generates the acceleration plots of latitude, 
    longitude, and altitude, requires latitude, longitude, altitude and 
    time to be in a pandas dataframe, with the columns labeled Latitude, 
    Longitude, Altitude, and Time respectively'''
    
    # gets acceleration data
    accel_data = process_flightsim_data(data)
    
    # plots the acceleration data
    fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True)
    fig.set_size_inches(18.5, 10.5)
    fig.suptitle('Acceleration of Latitude, Longitude, and Altitude vs Time')
    ax1.plot(accel_data.Time,accel_data.LatitudeAcceleration)
    ax1.set_ylabel('Latitude Acceleration (°)')
    ax2.plot(accel_data.Time,accel_data.LongitudeAcceleration)
    ax2.set_ylabel('Longitude Acceleration (°)')
    ax3.plot(accel_data.Time,accel_data.VerticalAcceleration)
    ax3.set_xlabel('Time')
    ax3.set_ylabel('Altitude Acceleration (ft)')
    plt.show()


flightsim_accel_plot(flightsim_mod)

In [ ]:
def process_piper_data(piper_data):
    ''' work in progress function: derives parameters from piper flight data'''
    piper_data_mod = piper_data.copy()
    for i in range(len(piper_data)-1):
        timeDiff = pd.to_timedelta(piper_data_mod.Timestamp[i+1]-piper_data_mod.Timestamp[i]).total_seconds()
    piper_data_mod['TimeDelta']= timeDiff
    
    altitudeDiff = piper_data_mod.Altitude-piper_data_mod.Altitude.shift(1)  
    vertSpeedDiff = altitudeDiff-altitudeDiff.shift(1)
    latitudeDiff = piper_data_mod.Latitude-piper_data_mod.Latitude.shift(1)
    latSpeedDiff = latitudeDiff-latitudeDiff.shift(1)
    longitudeDiff = piper_data_mod.Longitude-piper_data_mod.Longitude.shift(1)
    longSpeedDiff = longitudeDiff-longitudeDiff.shift(1)
    

    piper_data_mod['VerticalSpeed'] = altitudeDiff/piper_data_mod.TimeDelta
    piper_data_mod['VerticalAcceleration'] = vertSpeedDiff/piper_data_mod.TimeDelta
    piper_data_mod['LateralSpeed'] = latitudeDiff/piper_data_mod.TimeDelta
    piper_data_mod['LateralAcceleration'] = latSpeedDiff/piper_data_mod.TimeDelta
    piper_data_mod['LongitudalSpeed'] = longitudeDiff/piper_data_mod.TimeDelta
    piper_data_mod['LongitudalAcceleration'] = longSpeedDiff/piper_data_mod.TimeDelta
#     piper_data_mod['MagnitudeVelocity3D'] = math.sqrt((piper_data_mod.VerticalSpeed**2)+\
#                                                  (piper_data_mod.LateralSpeed**2)+\
#                                                  (piper_data_mod.LongitudalSpeed**2))
    
    print(piper_data_mod.head(10))
    return(piper_data_mod)
    timeSpan = piper_data_mod.Timestamp[61141]-piper_data_mod.Timestamp[1]
    frequency = timeSpan/61141
    
    print()
    print('FLight Duration:',timeSpan)
    print('Average Frequency:',frequency)

In [ ]:
process_piper_data(piper_table)

End of usable code

In [ ]:
#csv_df = pd.DataFrame(flightsim_table, columns=['Time', 'Longitude', 'Latitude'])
flightsim_table.set_index('Time')
flightsim_table.head(5)
flightsim_table.plot('Time', 'Longitude')


In [ ]:
json_table['Timestamp']
df = pd.DataFrame(json_table, columns = ['Timestamp', 'Speed'])
df.set_index("Timestamp")
print(df)
print(json_table.dtypes)
df.plot('Timestamp','Speed')


pd.plotting.register_matplotlib_converters() # needed for matplotlib-pandas bug

plt.plot(json_table.Timestamp, json_table.Altitude)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
pd.plotting.deregister_matplotlib_converters()
ax.plot(df['Timestamp'],               
       color='purple')

ax.set(xlabel='Time',
      ylabel='Speed',
      title='test')